In [8]:
import pandas as pd
bookings = pd.read_csv('bookings.csv', sep = ';')
bookings.head(7)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [7]:
bookings.shape

(119390, 21)

In [9]:
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [17]:
def to_lower_underscore(name):
    name = name.lower().replace(' ', '_')
    return name

In [18]:
bookings = bookings.rename(columns=to_lower_underscore)

-------------------------------------------------------------------------------

Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.

In [13]:
bookings.query('is_canceled == 0') \
        .country \
        .value_counts()[:5]

country
PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: count, dtype: int64

На сколько ночей в среднем бронируют отели разных типов?

In [23]:
bookings.groupby('hotel', as_index=False) \
        .agg({'stays_total_nights': 'mean'}) 

,hotel,stays_total_nights
0,City Hotel,2.978142
1,Resort Hotel,4.318547


Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?

In [24]:
bookings.query('assigned_room_type!=reserved_room_type').shape[0]

14917

Проанализируйте даты запланированного прибытия. 

– На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?

– Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов

In [25]:
bookings.query('arrival_date_year == 2016 and is_canceled == 0') \
        .arrival_date_month \
        .value_counts()

arrival_date_month
October      3689
May          3563
September    3372
April        3367
March        3347
August       3238
June         3196
July         3073
November     2818
February     2554
December     2462
January      1691
Name: count, dtype: int64

In [26]:
bookings.query('arrival_date_year == 2017 and is_canceled == 0') \
        .arrival_date_month \
        .value_counts()

arrival_date_month
May         3551
July        3329
March       3298
June        3208
April       3198
August      3109
February    2818
January     2431
Name: count, dtype: int64

In [42]:
bookings.query('hotel == "City Hotel" and is_canceled == 1') \
        .groupby('arrival_date_year') \
        .arrival_date_month \
        .value_counts() 


arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

Посмотрите на числовые характеристики трёх переменных: adults, children и babies. Какая из них имеет наибольшее среднее значение?

In [43]:
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

Создайте колонку total_kids, объединив children и babies. Отели какого типа в среднем пользуются большей популярностью у клиентов с детьми?

In [44]:
bookings['total_kids'] = bookings.children + bookings.babies

In [48]:
bookings.groupby('hotel')\
        .agg({'total_kids':'mean'})

,total_kids
hotel,
City Hotel,0.096311
Resort Hotel,0.142586


Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.

In [49]:
bookings['has_kids'] = bookings.total_kids > 0

In [50]:
no_kids_churn = bookings.query('is_canceled == 1 and has_kids == False').shape[0]/bookings.query('has_kids == False').shape[0]
no_kids_churn  = round(no_kids_churn  * 100, 2)
no_kids_churn 

37.22

In [51]:
yes_kids_churn = bookings.query('is_canceled == 1 and has_kids == True').shape[0]/bookings.query('has_kids == True').shape[0]
yes_kids_churn  = round(yes_kids_churn  * 100, 2)
yes_kids_churn 

34.92